# Task 2. RNA Sequences


In this task the Microbiome datasets will be treated. First it will be demonstrated how all nine spreadsheets containing RNA sequences can be loaded into the same Panda dataframe and later the Metadata will be used to create a more useful dataframe where the actual nature of the data is taken into account. 

Before being able to do anything it is nessary to take a look at the raw data. After a glance at the excel files it is evident that they all have to columns; one consisting of a RNA sequence and one with small integers. Since they all have the samae structure they are easily combined into a simple dataframe. It should be noted that we are still not taking any metadata into account.

The way this simple Panda daata frame will be constructed is by creating a vector of all the relevant filenames and afterwardds loop over the filenames and within the loop read the files into a temporary panda (rna2) and concatenate the temporary with the total dataframe (rna) one by one and still within the loop. We also give the index and the column names:

In [72]:
import pandas as pd
import numpy as np
filenames = ["MID1.xls","MID2.xls","MID3.xls","MID4.xls","MID5.xls","MID6.xls","MID7.xls","MID8.xls","MID9.xls"]
rna = pd.DataFrame()
rna2 = pd.DataFrame()
for filename in filenames:
    rna2 = pd.read_excel(filename, 'Sheet 1', index_col=0, header=None) 
    rna = pd.concat([rna,rna2])
rna.columns = ['Count']
rna.index.name = 'RNA-sequence'

In [73]:
rna.head()

,Count
RNA-sequence,
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera",7
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae Pyrolobus",2
"Archaea ""Crenarchaeota"" Thermoprotei Sulfolobales Sulfolobaceae Stygiolobus",3
"Archaea ""Crenarchaeota"" Thermoprotei Thermoproteales Thermofilaceae Thermofilum",3
"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Methanocellales Methanocellaceae Methanocella",7


The first five rows of the dataframe called rna is showed above. 

When working with data it is however important to understand it before a useful and relavant dataframe can be constructed. Therefore it is necessary to look at the metadata sheet and see how this aplies to the other sheets. We observe that:

 - NEC1 (perhaps one person or a group of people with a disease) has RNA sequences taken from the tissue and the stool. Data in MID2 and MID6
 - NEC2 is another group with the disease and RNA sequences taken from the tissue and the stool. Data in MID4 and MID8.
 - CONTROL1 and CONTROL2 are control groups for the two other groups and has the same tests (tissue and stool). Data in MID3, MID5, MID7, MID9.
 - Extraction control contains RNA sequences and results for unknown test (we dont know if this is from tissue or stool or a mix)
 
It will therefore now be demonstrated how the data will be put into a dataframe with the columns Group, RNA-sequence, Tissue, Stool and NA. The index will consist of both the group and the RNA-sequence.

Description of procedure:
 - Files with data concerning Tissue are first read into a panda and the column with numbers are given the name Tissue. These are all stored in a dictionary, so that they are easy to acess later.
 - Likewise files with data concerning Stool are read and the column is given the name Stool.
 - The file with NA data is read and the column is given this name
 - The Pandas with data from the same group is taken from the dictionary and concatenated column wise.
 - The five remaining dataframes are then concatenated while giving them the relevant keys according to their group.
 - The index is renamed, NaN' are replaced bu Unknown and the order of the columns is rearranged

In [74]:
filenames = ["MID1.xls","MID2.xls","MID3.xls","MID4.xls","MID5.xls","MID6.xls","MID7.xls","MID8.xls","MID9.xls"]
d = {} #dictionary created

for filename in filenames[1:5]: #reading files with tissue data
    d[filename] = pd.read_excel(filename, 'Sheet 1', index_col=0, header=None, names=['Tissue'])
    
for filename in filenames[5:9]: #reading files with stool data
    d[filename] = pd.read_excel(filename, 'Sheet 1', index_col=0, header=None, names=['Stool'])
    
d['MID1.xls'] = pd.read_excel(filename, 'Sheet 1', index_col=0, header=None, names=['NA']) #reading file with NA data

In [75]:
NEC1 = pd.concat([d['MID2.xls'],d['MID6.xls']],axis=1) #concatenating dataframes concerning group NEC1
Control1 = pd.concat([d['MID3.xls'],d['MID7.xls']],axis=1) #concatenating dataframes concerning group Control1
NEC2 = pd.concat([d['MID4.xls'],d['MID8.xls']],axis=1) #concatenating dataframes concerning group NEC2
Control2 = pd.concat([d['MID5.xls'],d['MID9.xls']],axis=1) #concatenating dataframes concerning group Control2
rna = pd.concat([NEC1,Control1,NEC2,Control2,d['MID1.xls']],keys=['NEC1','Control1','NEC2','Control2','Extraction Control']) #Concatenating all

In [81]:
rna.index.names = ['Group','RNA-sequence'] #Index names are changed
rna.replace(np.nan,'Unknown') #NaA replaced
rna = rna[['Tissue','Stool','NA']]  #Order of columns changed
rna

Tissue  \
Group              RNA-sequence                                                 
NEC1               Archaea "Crenarchaeota" Thermoprotei Acidilobal...     2.0   
                   Archaea "Crenarchaeota" Thermoprotei Acidilobal...    14.0   
                   Archaea "Crenarchaeota" Thermoprotei Desulfuroc...    23.0   
                   Archaea "Crenarchaeota" Thermoprotei Desulfuroc...     1.0   
                   Archaea "Crenarchaeota" Thermoprotei Desulfuroc...     2.0   
                   Archaea "Crenarchaeota" Thermoprotei Desulfuroc...     1.0   
                   Archaea "Crenarchaeota" Thermoprotei Desulfuroc...     NaN   
                   Archaea "Crenarchaeota" Thermoprotei Desulfuroc...     2.0   
                   Archaea "Crenarchaeota" Thermoprotei Sulfolobal...    10.0   
                   Archaea "Crenarchaeota" Thermoprotei Sulfolobal...    11.0   
                   Archaea "Crenarchaeota" Thermoprotei Thermoprot...     9.0   
                   Archaea "Crenarchaeota" Thermoprotei Thermoprot...     5.0   
                   Archaea "Crenarchaeota" Thermoprotei Thermoprot...     3.0   
                   Archaea "Euryarchaeota" "Methanomicrobia" Metha...     9.0   
                   Archaea "Euryarchaeota" "Methanomicrobia" Metha...     1.0   
                   Archaea "Euryarchaeota" "Methanomicrobia" Metha...    12.0   
                   Archaea "Euryarchaeota" "Methanomicrobia" Metha...     NaN   
                   Archaea "Euryarchaeota" "Methanomicrobia" Metha...     2.0   
                   Archaea "Euryarchaeota" Archaeoglobi Archaeoglo...     4.0   
                   Archaea "Euryarchaeota" Halobacteria Halobacter...     3.0   
                   Archaea "Euryarchaeota" Halobacteria Halobacter...     1.0   
                   Archaea "Euryarchaeota" Halobacteria Halobacter...     4.0   
                   Archaea "Euryarchaeota" Halobacteria Halobacter...     3.0   
                   Archaea "Euryarchaeota" Halobacteria Halobacter...     NaN   
                   Archaea "Euryarchaeota" Halobacteria Halobacter...     5.0   
                   Archaea "Euryarchaeota" Halobacteria Halobacter...     NaN   
                   Archaea "Euryarchaeota" Halobacteria Halobacter...     2.0   
                   Archaea "Euryarchaeota" Halobacteria Halobacter...    26.0   
                   Archaea "Euryarchaeota" Methanobacteria Methano...     1.0   
                   Archaea "Euryarchaeota" Methanopyri Methanopyra...    47.0   
...                                                                       ...   
Extraction Control Bacteria "Proteobacteria" Gammaproteobacteria O...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria O...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria O...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria P...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria P...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria P...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria P...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria P...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria P...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria P...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria T...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria T...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria T...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria T...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria X...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria X...     NaN   
                   Bacteria "Proteobacteria" Gammaproteobacteria X...     NaN   
                   Ba

We now have a complete dataframe where all information is collected and easy to read. Lets check that the index is in fact unique.

In [84]:
rna.index.is_unique

True

This is of course to be expected but it also confirms that dataframes have been concatenated correctly.